# Random Numbers (on a computer)
With material from http://www-static.etp.physik.uni-muenchen.de/kurs/comp20/ and https://henryiii.github.io/compclass/.

## Example 1: Buffon's needle
* “Buffonsches Nadelproblem” (Graf G.L.L. von Buffon, 1707 – 1788)
* draw parallel, equidistant lines (distance $d$), throw $N$ needles (length $l\leq d$)
* probability for needle intersecting with line: $p = l_\text{eff}/d = l|\cos \phi|/d$
* integrate, assuming flat distribution for angle $\phi$: 
$$p = \int_0^{2\pi} \frac {l \left|\cos {\varphi}\right|}{d} \frac {\mathrm d \varphi}{2\pi}=\frac {2l}{\pi d}$$
* von Mises: $N_\text{Treffer}/N \to p$ for $N\to\infty$ 
$$\tfrac{2\cdot N \cdot l}{N_\text{Treffer} \cdot d} \rightarrow \pi$$

What do you get for $\pi$ here?
![](../figures/Buffon_Streicholz_1.jpg)

## Example 2: area of a circle
* also here, we determine $\pi$, but this time from the numerical integration of the area of a circle
* approximate $\pi$ as ratio of hits $t$ and tries $n$:
  * $\lim_{n\to\infty} t/n = \pi / 4$ 
  * (because $A_\text{sector}=\pi r^2 / 4$ and $A_\square = r^2$)

How to define a circle:
* circle = shape consisting of all points in a plane at a given distance from the center
* unit circle, using Euclidean distance in 2-D: $x^2 + y^2 \leq 1$
Implementation:

In [ ]:
import numpy as np
xy    = np.random.rand(2, 10000)
valid = np.sum(xy ** 2, axis=0) < 1
good  = xy[:, valid]
bad   = xy[:, ~valid]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(*good, ".")
plt.plot(*bad, ".")
plt.axis("equal");

In [ ]:
valid

In [ ]:
np.mean(valid) * 4

Can easily be generalized to higher dimensions:
* e.g. for a sphere: $x^2+y^2+z^2 \leq 1$
* N dimensions: $\sum_{i=1}^N x_i^2 \leq 1$

Volume of a sphere:

In [ ]:
xyz   = np.random.rand(3, 10000)
valid = np.sum(xyz ** 2, axis=0) < 1
good  = xyz[:, valid]
bad   = xyz[:, ~valid]
print("MC:      ", np.mean(valid)*8) # explain: why MC?
print("Analytic:", 4 / 3 * np.pi)

## Generating random numbers
We have seen that random numbers are useful / necessary to do (Monte Carlo) integrations on a computer. What is behind the `np.rand` function, i.e. how can we construct random numbers using a deterministic system like a computer?

* first task: we need a generator for random numbers
  * will then see that we can generate random numbers following arbitrary distributions (Gaussian, Poissonian, exponential, ...) from this
* on a deterministic computer, we can only generate *pseudo-random numbers*
  * special hardware for true random numbers (important for secure cryptography: attacker must not be able to infer random numbers) 
* we want these *pseudo-random numbers* to be
  * independently and identically distributed (i.i.d.) 
  * have a long period
  * fast to compute, easy on memory
  * reproducible (for debugging)
* pseudo-random numbers with these properties are suitable for our purposes (e.g. MC integration)

### Simple random-number generator: [linear congruential generator](https://en.wikipedia.org/wiki/Linear_congruential_generator)
* recursive definition: $I_j=(a\cdot I_{j-1}+c) \mod m$ 
* 3 integer constants: multiplier $a$, increment $c$, modulus $m$ 
* plus: start (or seed) value $I_0$ (*random seed*)
* generates sequence $I_1,I_2,...$ with $0 \leq I_j \leq m-1$
  * $I_j$ is periodic sequence with (maximum) period $m$
  * $u_j = I_j/m \in [0, 1)$

In [ ]:
def lin_cong_iter(c, a, m, I_0):
    """implements a linear congruential generator"""
    I_j = I_0

    while True:
        yield I_j # <- Python generator
        I_j = (a * I_j + c) % m
        if I_j == I_0:
            # arrived at seed value again
            yield I_j
            break

In [ ]:
list(lin_cong_iter(0, 3, 7, 1))

Map to range $[0,1)$:

In [ ]:
list(i / 7 for i in lin_cong_iter(0, 3, 7, 1))

Do these look random? Can you predict the next number from the previous numbers without knowing the implementation?

### Testing randomness
* correlations will spoil MC computations, results will be biased (i.e. wrong)
* define suite of tests (e.g. [TestU01](https://en.wikipedia.org/wiki/TestU01)) that random-number generators have to pass, e.g.
  * test flatness of distributon ($\chi^2$ tests for sub-intervals of $[0,1)$)
  * correlation tests ([spectral tests](https://en.wikipedia.org/wiki/Spectral_test))
  
Let's do a spectral test of our LCG:

In [ ]:
# generate some random numbers
lcg_numbers = list(lin_cong_iter(a=57, c=1, m=256, I_0=10))[:-1]
print(len(lcg_numbers))

In [ ]:
# numpy's random numbers
real_rand = np.random.randint(low = 0, high = 256, size = len(lcg_numbers))

In [ ]:
# plot sequence
fig, axs = plt.subplots(1, 2, figsize=(16, 4))
axs[0].plot(range(len(lcg_numbers)), lcg_numbers, "x-", label = "lcg_numbers")
axs[0].legend()
axs[1].plot(range(len(lcg_numbers)), real_rand, "x-", label = "numpy")
axs[1].legend()
plt.show()

No visible difference.

In [ ]:
# flatness
plt.hist(lcg_numbers, bins = 20, alpha = 0.5, label = "lcg_numbers")
plt.hist(real_rand, bins = 20, alpha = 0.5, label = "numpy")
plt.legend()
plt.show()

## 🤔

In [ ]:
# spectral test in 2-D
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].plot(lcg_numbers[::2], lcg_numbers[1::2], "+", label = "lcg_numbers")
axs[0].legend()
axs[1].plot(real_rand[::2], real_rand[1::2], "+", label = "numpy")
axs[1].legend()
plt.show()

## 😲

# Monte Carlo methods